In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000352815s, CPU 0.000341s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00623288s, CPU 0.006221s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetric)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)


LowLevelFEM.Problem("body1-2D", :AxiSymmetric, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [4]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [5]:
u0 = FEM.initialDisplacement(problem, "body", ux=0, uy=0)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :u2D)

In [6]:
FEM.initialDisplacement!(problem, "left", u0, ux=0, uy=0)
u0

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :u2D)

In [7]:
v0 = FEM.initialVelocity(problem, "body", vx=0, vy=0)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :v2D)

In [8]:
FEM.initialVelocity!(problem, "body", v0, vx=0, vy=0)

In [9]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f2D)

In [10]:
FEM.applyBoundaryConditions!(problem, K, M, f, [supp])

In [11]:
Tmin = FEM.smallestPeriodTime(K, M)

4.282263524466644e-8

In [12]:
u, v = FEM.CDM(K, M, f, u0, v0, 10Tmin, Tmin / π)

(LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 3.40758171563843e-6 3.1261420083258764e-6; 0.0 0.0 … -1.5087603788237494e-6 -1.3955634838544426e-6], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.0904694520655806e-7, 1.226778133573778e-7  …  2.998790993180345e-7, 3.135099674688542e-7, 3.2714083561967397e-7, 3.407717037704937e-7, 3.5440257192131345e-7, 3.680334400721332e-7, 3.8166430822295294e-7, 3.952951763737727e-7, 4.0892604452459243e-7, 4.225569126754122e-7], Int64[], 32, :u2D), LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … -26.75648630602364 -20.647232751321702; 0.0 0.0 … 12.481664988117526 8.304452344255063], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.

In [13]:
FEM.showDoFResults(problem, u, :uvec, visible=true)

1

In [14]:
FEM.showDoFResults(problem, v, :vvec)

2

In [15]:
e1 = FEM.solveStrain(problem, u)

LowLevelFEM.TensorField([[0.0 0.0 … 3.57608572343763e-6 3.577639228835607e-6; 0.0 0.0 … 1.3817193199166863e-6 6.883729177632393e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 5.263940016607355e-6 4.790102602245689e-6], [0.0 0.0 … 4.831474527863377e-6 4.274951580947013e-6; 0.0 0.0 … 6.04614866903533e-7 7.477052500653222e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.9061300755388115e-6 4.402881159260814e-6], [0.0 0.0 … 4.53650042620972e-6 5.1407103395128625e-6; 0.0 0.0 … -6.750544137103238e-7 -6.987529555873933e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 5.568392730303157e-6 5.005411762320317e-6], [0.0 0.0 … 6.932040936330109e-6 6.902503887603962e-6; 0.0 0.0 … -3.602120785082629e-7 -4.6196337892997834e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 6.014771259310531e-6 5.191691538754419e-6], [0.0 0.0 … 5.090681876932699e-6 5.7516169128474595e-6; 0.0 0.0 … -7.306656924872101e-7 -7.036379353207167e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 5.710490628360014e-6 5.349746136948725e-6], [0.0 0.0 … 5.040470602467273e-6 4.7582282617826

In [16]:
FEM.showStrainResults(problem, e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [17]:
e2 = FEM.solveStrain(problem, u, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 5.562425094081304e-6 5.500992492917056e-6; 0.0 0.0 … 1.6935417361111928e-5 1.742341059784024e-5; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.549724385547846e-6 4.173952590099804e-6], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.0904694520655806e-7, 1.226778133573778e-7  …  2.998790993180345e-7, 3.135099674688542e-7, 3.2714083561967397e-7, 3.407717037704937e-7, 3.5440257192131345e-7, 3.680334400721332e-7, 3.8166430822295294e-7, 3.952951763737727e-7, 4.0892604452459243e-7, 4.225569126754122e-7], Int64[], 32, :e)

In [18]:
FEM.showDoFResults(problem, e2, :e)

4

In [19]:
e3 = FEM.elementsToNodes(problem, e1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 5.562425094081304e-6 5.500992492917056e-6; 0.0 0.0 … 8.467708680555964e-6 8.71170529892012e-6; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.549724385547846e-6 4.173952590099804e-6], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.0904694520655806e-7, 1.226778133573778e-7  …  2.998790993180345e-7, 3.135099674688542e-7, 3.2714083561967397e-7, 3.407717037704937e-7, 3.5440257192131345e-7, 3.680334400721332e-7, 3.8166430822295294e-7, 3.952951763737727e-7, 4.0892604452459243e-7, 4.225569126754122e-7], Int64[], 32, :e)

In [20]:
FEM.showDoFResults(problem, e3, :e)

5

In [21]:
s1 = FEM.solveStress(problem, u)

LowLevelFEM.TensorField([[0.0 0.0 … 1.091924620261126 1.1538754426017086; 0.0 0.0 … 0.21257220306410557 0.10590352580972913; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.3928802739489925 1.3850876665476684], [0.0 0.0 … 1.408302161990219 1.1787827110438363; 0.0 0.0 … 0.09301767183131278 0.11503157693312649; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.378501868432377 1.153784787363089], [0.0 0.0 … 1.5165964377586532 1.6203085259009415; 0.0 0.0 … -0.10385452518620365 -0.10750045470575281; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.6589531886126214 1.5597624707988105], [0.0 0.0 … 2.0290695433129953 1.7991884362329733; 0.0 0.0 … -0.05541724284742506 -0.07107128906615051; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.9613100729768422 1.7250848481822225], [0.0 0.0 … 1.4484825572125886 1.5453751029767138; 0.0 0.0 … -0.11241010653649386 -0.10825199004934104; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.5602336610541205 1.5232797191417247], [0.0 0.0 … 1.7256740573444795 1.4631754722826091; 0.0 0.0 … 0.1094090731050147 0.12523274432354614; … ;

In [22]:
FEM.showStressResults(problem, s1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


6

In [23]:
s2 = FEM.solveStress(problem, u, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 1.396194587953816 1.410741566200325; 0.0 0.0 … 1.3027244123932253 1.3402623536800182; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.1406984063795855 1.1358916680340008], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.0904694520655806e-7, 1.226778133573778e-7  …  2.998790993180345e-7, 3.135099674688542e-7, 3.2714083561967397e-7, 3.407717037704937e-7, 3.5440257192131345e-7, 3.680334400721332e-7, 3.8166430822295294e-7, 3.952951763737727e-7, 4.0892604452459243e-7, 4.225569126754122e-7], Int64[], 32, :s)

In [24]:
FEM.showDoFResults(problem, s2, :s)

7

In [25]:
err1 = FEM.fieldError(problem, e1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 1.139491915352784e-6 2.47739758183919e-7; 0.0 0.0 … 1.3042656238140205e-7 1.807412424432008e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.0904694520655806e-7, 1.226778133573778e-7  …  2.998790993180345e-7, 3.135099674688542e-7, 3.2714083561967397e-7, 3.407717037704937e-7, 3.5440257192131345e-7, 3.680334400721332e-7, 3.8166430822295294e-7, 3.952951763737727e-7, 4.0892604452459243e-7, 4.225569126754122e-7], Int64[], 32, :e)

In [26]:
FEM.showDoFResults(problem, err1, :tensor)

8

In [27]:
u2, v2 = FEM.HHT(K, M, f, u0, v0, 10Tmin, Tmin / π)

(LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.3801357636609307e-9 … 3.541791841799697e-6 3.250897387589948e-6; 0.0 -1.636426848298791e-10 … -1.583505378819221e-6 -1.4278603765010996e-6], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8, 6.815434075409879e-8, 8.178520890491854e-8, 9.54160770557383e-8, 1.0904694520655806e-7, 1.226778133573778e-7  …  2.998790993180345e-7, 3.135099674688542e-7, 3.2714083561967397e-7, 3.407717037704937e-7, 3.5440257192131345e-7, 3.680334400721332e-7, 3.8166430822295294e-7, 3.952951763737727e-7, 4.0892604452459243e-7, 4.225569126754122e-7], Int64[], 32, :u2D), LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.20250152057672569 … -19.86609010896359 -22.815630672299314; 0.0 -0.024010603436148366 … 9.41497380465488 13.422163286831255], [0.0, 1.3630868150819757e-8, 2.7261736301639515e-8, 4.089260445245927e-8, 5.452347260327903e-8

In [28]:
FEM.showDoFResults(problem, u2, :uvec)

9

In [29]:
FEM.showDoFResults(problem, v2, :vvec)

10

In [30]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [31]:
gmsh.finalize()